In [23]:
import pandas as pd
dfcrsp = pd.read_csv("crsp_sample.csv")
#preload
dfann = pd.read_csv("announcements_sample.csv")

In [24]:
print(dfcrsp.head())
print(dfann.head())


   Unnamed: 0  permno        date       ret  shrcd       price            me
0           0   10026  2021-01-29 -0.017442     11  152.660004  2.897487e+06
1           1   10026  2021-02-26  0.039958     11  158.759995  3.013265e+06
2           2   10026  2021-03-31 -0.007275     11  157.029999  2.988909e+06
3           3   10026  2021-04-30  0.048271     11  164.610001  3.133516e+06
4           4   10026  2021-05-28  0.066642     11  175.580002  3.342341e+06
   Unnamed: 0        date fiscal_period  permno  gics_6d  ret_ex_3d  \
0          64  2021-01-25    2020-12-31   10026   302020   0.085726   
1          65  2021-04-26    2021-03-31   10026   302020  -0.040383   
2          66  2021-07-26    2021-06-30   10026   302020   0.005235   
3          67  2021-11-15    2021-09-30   10026   302020  -0.015813   
4         123  2021-01-20    2020-12-31   10032   452030  -0.072586   

   date_diff  doy_diff  
0          0        -2  
1          0        -2  
2          0        -2  
3          

1) How many stock-months (rows) are there in the crsp dataframe? How many stock-quarters (rows) are there in announcements.csv?


In [25]:
print(dfcrsp.shape[0].__str__()+" is how many stock months there are")
#gives how many rows are in the df for crsp
print(dfann.shape[0].__str__()+" is how many stock quarters we have")


104664 is how many stock months there are
9146 is how many stock quarters we have


2) How many unique permnos are there in the announcements dataframe? Hint: there is a “unique” method. Use “len” also.

In [26]:
num_unique_entries = dfann["permno"].nunique()

print(num_unique_entries)
# separate version that Kerry probably wrote.
print('there are '+len(dfann['permno'].unique()).__str__()+" unique permnos")

2509
there are 2509 unique permnos


3)    Create a “quarter” column in the announcements dataframe based on the fiscal_period column.

In [27]:


dfann['Quarter'] = pd.to_datetime(dfann['date']).dt.to_period('Q')
dfann['Quarter'].head(30)



0     2021Q1
1     2021Q2
2     2021Q3
3     2021Q4
4     2021Q1
5     2021Q2
6     2021Q3
7     2021Q4
8     2021Q1
9     2021Q2
10    2021Q3
11    2021Q4
12    2021Q1
13    2021Q2
14    2021Q3
15    2021Q4
16    2021Q1
17    2021Q2
18    2021Q3
19    2021Q4
20    2021Q1
21    2021Q2
22    2021Q3
23    2021Q4
24    2021Q1
25    2021Q2
26    2021Q3
27    2021Q4
28    2021Q1
29    2021Q2
Name: Quarter, dtype: period[Q-DEC]

4) Create a “quarter” column in the crsp dataframe based on the date column.

In [28]:
dfcrsp['Quarter'] = pd.to_datetime(dfcrsp['date']).dt.to_period('Q')

#prep code for next part

#dfcrsp['Year'] = pd.to_datetime(dfcrsp['date']).dt.year
print(dfcrsp['Quarter'].head(30))

#print(dfcrsp['Year'].head(30))

0     2021Q1
1     2021Q1
2     2021Q1
3     2021Q2
4     2021Q2
5     2021Q2
6     2021Q3
7     2021Q3
8     2021Q3
9     2021Q4
10    2021Q4
11    2021Q4
12    2021Q1
13    2021Q1
14    2021Q1
15    2021Q2
16    2021Q2
17    2021Q2
18    2021Q3
19    2021Q3
20    2021Q3
21    2021Q4
22    2021Q4
23    2021Q4
24    2021Q1
25    2021Q1
26    2021Q1
27    2021Q2
28    2021Q2
29    2021Q2
Name: Quarter, dtype: period[Q-DEC]


5)Extract the last day of each quarter for each permno in the crsp dataframe. Hint: use groupby and .last().

In [29]:
lastdayquarter = dfcrsp.groupby(['permno','Quarter']).last()
lastdayquarter.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 35597 entries, (10026, Period('2021Q1', 'Q-DEC')) to (93436, Period('2021Q4', 'Q-DEC'))
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  35597 non-null  int64  
 1   date        35597 non-null  object 
 2   ret         34855 non-null  float64
 3   shrcd       35597 non-null  int64  
 4   price       35507 non-null  float64
 5   me          35597 non-null  float64
dtypes: float64(3), int64(2), object(1)
memory usage: 1.8+ MB


6)Change the name of the ‘date’ column in the announcements dataframe to ‘ea_date’. This is the date of the firm’s quarterly earnings announcement. Hint: pass a dictionary as columns = {‘old name’: ‘new name’} to the rename method.

In [30]:
dfann = dfann.rename(columns={'date':'ea_date'})
dfann.head()

,Unnamed: 0,ea_date,fiscal_period,permno,gics_6d,ret_ex_3d,date_diff,doy_diff,Quarter
0,64,2021-01-25,2020-12-31,10026,302020,0.085726,0,-2,2021Q1
1,65,2021-04-26,2021-03-31,10026,302020,-0.040383,0,-2,2021Q2
2,66,2021-07-26,2021-06-30,10026,302020,0.005235,0,-2,2021Q3
3,67,2021-11-15,2021-09-30,10026,302020,-0.015813,7,9,2021Q4
4,123,2021-01-20,2020-12-31,10032,452030,-0.072586,0,-2,2021Q1


7)    Merge the reduced crsp dataframe with the announcement dataframe, matching on [‘permno’, ‘quarter’]. Hint: permno and quarter will form the index of the reduced crsp dataframe after grouping. You will need to reset the index or use left_on=… and right_index=True, or right_on=… and left_index=True in the merge.


In [31]:
mergedf = lastdayquarter.merge(dfann, on=['permno', 'Quarter'], how='right')

mergedf.head()
mergedf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9146 entries, 0 to 9145
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype        
---  ------         --------------  -----        
 0   permno         9146 non-null   int64        
 1   Quarter        9146 non-null   period[Q-DEC]
 2   Unnamed: 0_x   9137 non-null   float64      
 3   date           9137 non-null   object       
 4   ret            9136 non-null   float64      
 5   shrcd          9137 non-null   float64      
 6   price          9136 non-null   float64      
 7   me             9137 non-null   float64      
 8   Unnamed: 0_y   9146 non-null   int64        
 9   ea_date        9146 non-null   object       
 10  fiscal_period  9146 non-null   object       
 11  gics_6d        9146 non-null   int64        
 12  ret_ex_3d      9115 non-null   float64      
 13  date_diff      9146 non-null   int64        
 14  doy_diff       9146 non-null   int64        
dtypes: float64(6), int64(5), object(3), pe

8)    Filter the merged dataframe to keep only rows satisfying price>=5, me>=50000, and shrcd in [10, 11]. Save this as a csv file with the name merged.csv and submit on Canvas. Use the filtered dataframe to answer all remaining questions.


In [32]:
# Define conditions for dropping rows
condition_A = mergedf['price'] >= 5
condition_B = mergedf['me'] >= 50000
condition_C = mergedf['shrcd'].isin([10, 11])

# Create a combined boolean mask to drop rows
combined_condition = condition_A & condition_B & condition_C

# Drop rows that meet the specified conditions
df_filtered = mergedf[combined_condition]
df_filtered.to_csv("merged.csv")
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6838 entries, 0 to 9145
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype        
---  ------         --------------  -----        
 0   permno         6838 non-null   int64        
 1   Quarter        6838 non-null   period[Q-DEC]
 2   Unnamed: 0_x   6838 non-null   float64      
 3   date           6838 non-null   object       
 4   ret            6838 non-null   float64      
 5   shrcd          6838 non-null   float64      
 6   price          6838 non-null   float64      
 7   me             6838 non-null   float64      
 8   Unnamed: 0_y   6838 non-null   int64        
 9   ea_date        6838 non-null   object       
 10  fiscal_period  6838 non-null   object       
 11  gics_6d        6838 non-null   int64        
 12  ret_ex_3d      6830 non-null   float64      
 13  date_diff      6838 non-null   int64        
 14  doy_diff       6838 non-null   int64        
dtypes: float64(6), int64(5), object(3), period[

9)    How many stock-quarters are there in the filtered dataframe and how many unique permnos?

In [33]:
print(df_filtered.shape[0].__str__()+" is how many stock quarters there are")

# separate version that Kerry probably wrote.
                                     

print('there are '+len(df_filtered['permno'].unique()).__str__()+" unique permnos")
from datetime import datetime


6838 is how many stock quarters there are
there are 1853 unique permnos


10)    Which day of the week is the most popular day for earnings announcements?
What fraction of earnings announcements occur on this day?
 Hint: the weekday method of a datetime object returns 0 for Monday, 1 for Tuesday, etc.

In [34]:
df_filtered['ea_date'] = pd.to_datetime(df_filtered['ea_date'])
df_filtered.loc[:,'weekday']=df_filtered['ea_date'].dt.day_name()
daycounts = df_filtered['weekday'].value_counts()
print(daycounts)
maxfrac = daycounts.max()/daycounts.sum()
print("the total amount of days is "+daycounts.sum().__str__()+" Thursday accounts for " + (maxfrac.__round__(4)*100).__str__()+"% of all announcements")


weekday
Thursday     2357
Wednesday    1793
Tuesday      1425
Monday        836
Friday        427
Name: count, dtype: int64
the total amount of days is 6838 Thursday accounts for 34.47% of all announcements


C:\Users\rra3\AppData\Local\Temp\ipykernel_5560\1114864487.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['ea_date'] = pd.to_datetime(df_filtered['ea_date'])
C:\Users\rra3\AppData\Local\Temp\ipykernel_5560\1114864487.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered.loc[:,'weekday']=df_filtered['ea_date'].dt.day_name()



11)    For each permno, find the most popular day of the week for earnings announcements. For each permno, calculate the fraction of the announcements that are made on that day. Save this as a csv file with the name days.csv and submit on Canvas. What is the mean of this fraction across permnos?

In [35]:
import numpy as np
counters2 = df_filtered.groupby(['permno', 'weekday']).size().reset_index(name='count')
most_popular_weekday_df = counters2.loc[counters2.groupby('permno')['count'].idxmax()].reset_index(drop=True)
total_announcements_per_permno = df_filtered.groupby('permno').size().reset_index(name='total_count')
most_popular_weekday_df = pd.merge(most_popular_weekday_df, total_announcements_per_permno, on='permno')
most_popular_weekday_df['fraction'] = most_popular_weekday_df['count'] / most_popular_weekday_df['total_count']
final_dataset = pd.merge(df_filtered, most_popular_weekday_df[['permno', 'weekday', 'fraction']], on='permno', how='left')
final_dataset = final_dataset.rename(columns={"weekday_x": "weekday", "weekday_y": "most_popular_weekday", "fraction": "fraction_on_most_popular_weekday"})
df_filtered.to_csv("days.csv")
# The mean of this fraction across 'permno': 

#mean_fraction_per_permno = final_dataset.groupby('fraction_on_most_popular_weekday').mean()
mean_fraction_per_permno = final_dataset.groupby('permno')['fraction_on_most_popular_weekday'].mean()
mean_of_means = mean_fraction_per_permno.mean()
print(f'The mean of this fraction across permnos is {np.mean(mean_of_means)}.')

The mean of this fraction across permnos is 0.7800413743479043.


 12)   Cut the dataframe into ‘above median’ and ‘below median’ based on the absolute value of date_diff. What is the mean of the fraction in the previous queston across permnos within each of the two groups?

In [36]:
days_df=final_dataset

days_df['quantile_group'] = pd.qcut(days_df['date_diff'].abs(), 2, labels=['below_median', 'above_median'])
AboveBelow = days_df.groupby(['permno','quantile_group']).fraction_on_most_popular_weekday.mean().unstack('permno').T
below = AboveBelow['below_median'].mean().__round__(3)*100
above = AboveBelow['above_median'].mean().__round__(3)*100
print(f'For the below median group, the mean fraction is approximately {below}%.')
print(f'For the above median group, the mean fraction is approximately {above}%.')

For the below median group, the mean fraction is approximately 77.8%.
For the above median group, the mean fraction is approximately 75.0%.


C:\Users\rra3\AppData\Local\Temp\ipykernel_5560\2537444102.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  AboveBelow = days_df.groupby(['permno','quantile_group']).fraction_on_most_popular_weekday.mean().unstack('permno').T


13)    For each permno, calculate the fraction of zeros for date_diff.

In [37]:
grouped_df = days_df.groupby('permno').date_diff.apply(lambda x: (x == 0).sum() / len(x)).reset_index()
grouped_df.columns = ['permno', 'fraction_of_zeros']
merged_df = pd.merge(days_df, grouped_df, on='permno', how='left')


14)    Filter the dataframe further to include only stock-quarters for which

        the fraction of zeros for date_diff is less than 0.5

        the fraction of announcements on the permno’s most popular day of the week is less than 0.9.

In [40]:
condition14=(merged_df['fraction_of_zeros']<0.5)&(merged_df['fraction_on_most_popular_weekday']<0.9)
df_uglyconditional=merged_df[condition14]
df_uglyconditional['fraction_of_zeros'].describe()
df_uglyconditional['fraction_on_most_popular_weekday'].describe()

df_uglyconditional.to_csv("filtered.csv")


15) Save this as a csv file with the name filtered.csv and submit on Canvas.

    How many stock-quarters are in the newly filtered dataframe? How many unique permnos?

In [39]:
print(df_uglyconditional.shape[0].__str__()+" is how many stock quarters there are")

# separate version that Kerry probably wrote.
                                     

print('there are '+len(df_uglyconditional['permno'].unique()).__str__()+" unique permnos")


2227 is how many stock quarters there are
there are 591 unique permnos
